###### Project

In [ ]:
import pandas as pd
#import sqlite3
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import datetime as dt
import folium
from folium import plugins
import geopandas
from calendar import monthrange

spray_data = pd.read_csv(
    'Files/spray.csv',  # шлях до файлу, який містить дані
    sep=',',  # роздільник стовпців
    header=0  # номер рядка, що містить заголовок (якщо заголовка немає, можна передати None)
)

train_data = pd.read_csv(
    'Files/train.csv',  # шлях до файлу, який містить дані
    sep=',',  # роздільник стовпців
    
    header=0  # номер рядка, що містить заголовок (якщо заголовка немає, можна передати None)
)

weather_data = pd.read_csv(
    'Files/weather.csv',  # шлях до файлу, який містить дані
    sep=',',  # роздільник стовпців
    header=0  # номер рядка, що містить заголовок (якщо заголовка немає, можна передати None)
)

test_truncated_data = pd.read_excel(
    'Files/test_truncated.xlsx',
    sheet_name='test_truncated',  # прочитати всі листи
    usecols="A:J"
    #nrows=5
)
#test_truncated_data

In [ ]:
def add_datetime(train_data):
    train_data['day']=pd.to_datetime(train_data.Date).dt.day
    train_data['weekday']=pd.to_datetime(train_data.Date).dt.weekday
    train_data['month']=pd.to_datetime(train_data.Date).dt.month
    train_data['year']=pd.to_datetime(train_data.Date).dt.year
    return train_data

def bar_chat(_data, grop_list_, agr_colum, agr_func):
    grop_list=[grop_list_]
    agg_func_math = {
        agr_colum: [agr_func]
    }
    plot_mas = _data.groupby(grop_list, group_keys=True, dropna=True).agg(agg_func_math).to_dict()[(agr_colum, agr_func)] 
    fig = plt.figure()
    plt.bar(plot_mas.keys(), plot_mas.values())
    plt.title(f'{grop_list_} {agr_colum} {agr_func} chart')    
    plt.grid(True)   # лінії допоміжної сітки
#bar_chat(test_truncated_data, 'year', 'year', 'count')

def group_data(_data, grop_list_, agr_colum, agr_func):
    grop_list=[grop_list_]
    agg_func_math = {
        agr_colum: [agr_func]
    }
    dict_data=_data.groupby(grop_list, group_keys=True, dropna=True).agg(agg_func_math).to_dict()[(agr_colum, agr_func)]
    df=pd.DataFrame(list(dict_data. items ()), columns = [grop_list_, agr_colum])
    #df.set_index(df[grop_list], *, drop=True, append=False, inplace=False, verify_integrity=False)[source]
    df.set_index(grop_list, inplace=True)
    return df.copy()

def concat_train_data(train_data):
    Trap=group_data(train_data, 'Trap', 'Latitude', 'min')
    Longitude=group_data(train_data, 'Trap', 'Longitude', 'min')
    NumMosquitos=group_data(train_data, 'Trap', 'NumMosquitos', 'sum')
    WnvPresent=group_data(train_data, 'Trap', 'WnvPresent', 'sum')
    AddressAccuracy=group_data(train_data, 'Trap', 'AddressAccuracy', 'min')
    RESTUANS=group_data(train_data, 'Trap', 'RESTUANS', 'sum')
    PIPIENS=group_data(train_data, 'Trap', 'PIPIENS', 'sum')
    Date_Min=group_data(train_data, 'Trap', 'Date', 'min')
    #Date_Max=group_data(train_data, 'Trap', 'Date', 'max')
    
    frames = [Trap, Longitude, Date_Min, NumMosquitos, WnvPresent, AddressAccuracy, RESTUANS, PIPIENS]
    result = pd.concat(frames, axis=1)
    result.reset_index(inplace=True)
    return result.copy()
#train_data
#df=concat_train_data(train_data[(train_data['year']==2013)])
#df['test']='2021'
#df
#weather_data
def make_category(X_Data, list_category):
    for i in list_category:
        X_Data['Cat_category']=X_Data[i].astype("category")
        clear_data(X_Data, [i], revers=False)
        print(i)
        X_Data.rename(
            columns={'Cat_category': i,  # колонка "SibSp" буде перейменована на "SiblingsSpouses"
                    }, inplace=True  # покажчик того, що зміни вносяться до існуючої таблиці
        )
    return X_Data

def clear_data(test_data, drop_list, revers):
    columns_df=[column for column in test_data]
    #print(columns_df)
    if revers==False:
        for i in drop_list:
            if i in columns_df:
                test_data.drop(
                    i,  # список колонок або рядків, які потрібно видалити
                    axis=1,  # видалення рядків відбувається аналогічно, щоб видалити саме колонки, вибираємо відповідну вісь
                    inplace=True  # видалення "на місці", без надання нової змінної
                )
    
        
    if revers==True:
        for i in columns_df:
            if i in drop_list:
                pass
                #print(i)
            else:
                #print(i)
                test_data.drop(
                    i,  # список колонок або рядків, які потрібно видалити
                    axis=1,  # видалення рядків відбувається аналогічно, щоб видалити саме колонки, вибираємо відповідну вісь
                    inplace=True  # видалення "на місці", без надання нової змінної)
                )
                
            
    return test_data.copy()

def replase_M_to_Max_int(weather_data, replase_column, date_type):
    #df=weather_data.copy()
    try:
        weather_data[replase_column].unique()
        weather_data[replase_column]=weather_data[replase_column].replace(regex=['M'], value='-1000').astype(str).astype(date_type)
        
        M=weather_data[replase_column].max()
        
        weather_data.loc[(weather_data[replase_column] ==-1000), replase_column] = M
        print(f'{replase_column}_{M} {weather_data[replase_column].unique()}')
    except Exception as _ex:
        print(f"[{replase_column}] Error", _ex)
    return weather_data.copy()

In [ ]:
#weather_data

weather_data=clear_data(weather_data, ['Water1'], revers=False)
list_category=['CodeSum']
weather_data=make_category (weather_data, list_category)    

#Depth_2
replase_column='Depth'
weather_data[replase_column].unique()
level_map = {'0': 0, 'M': 1}
weather_data[replase_column] = weather_data[replase_column].map(level_map).astype(str).astype(int)
#weather_data[replase_column].unique()


#SnowFall
replase_column='SnowFall'
level_map = {'0.0': 0, 'M': 0.5, '  T':1,  '0.1':0.1}
weather_data[replase_column] = weather_data[replase_column].map(level_map).astype(str).astype(float)
weather_data[replase_column].unique()

#Sunrise
replase_column='Sunrise'
weather_data[replase_column] =weather_data[replase_column] .replace(regex=['-'],value='0').astype(str).astype(int)
#weather_data[replase_column].unique()

#Sunset
replase_column='Sunset'
weather_data[replase_column] =weather_data[replase_column] .replace(regex=['-'],value='0').astype(str).astype(int)
#weather_data[replase_column].unique()

#PrecipTotal
replase_column='PrecipTotal'
weather_data[replase_column] =weather_data[replase_column] .replace(regex=['M'],value='-1000')
M=weather_data[replase_column].max()
weather_data.loc[(weather_data[replase_column] =='-1000'), replase_column] = M
print (M)



weather_data[replase_column] =weather_data[replase_column] .replace(regex=['  T'],value='0')
'''T=weather_data[replase_column].min()
print (f'{T}_{weather_data.loc[(weather_data[replase_column] ==1000), replase_column]}')
weather_data.loc[(weather_data[replase_column] ==1000), replase_column] = weather_data[replase_column].min()'''
weather_data['1']=weather_data[replase_column].astype(str).astype(float)


clear_data(weather_data, [replase_column], revers=False)

weather_data.rename(
            columns={'1': replase_column,  # колонка "SibSp" буде перейменована на "SiblingsSpouses"
                    }, inplace=True  # покажчик того, що зміни вносяться до існуючої таблиці
        )

#weather_data[replase_column].unique()
weather_data=replase_M_to_Max_int(weather_data, 'Heat', int)
weather_data=replase_M_to_Max_int(weather_data, 'Cool', int)
weather_data=replase_M_to_Max_int(weather_data, 'Depart', int)
weather_data=replase_M_to_Max_int(weather_data, 'Tavg', int)
weather_data=replase_M_to_Max_int(weather_data, 'WetBulb', int)
weather_data=replase_M_to_Max_int(weather_data, 'StnPressure', float)
weather_data=replase_M_to_Max_int(weather_data, 'AvgSpeed', float)
weather_data=replase_M_to_Max_int(weather_data, 'SeaLevel', float)
weather_data=add_datetime(weather_data)

In [ ]:
#train_data

from datetime import datetime, timedelta

def iter_days(draft_year, draft_month):
    dt = datetime(draft_year, draft_month, 1)
    while dt.year == draft_year and  dt.month == draft_month:
        yield dt
        dt += timedelta(days=1)


train_data=add_datetime(train_data)
train_data['RESTUANS']=train_data['Species'].str.contains('True').astype(str).astype(bool)
train_data['PIPIENS']=train_data['Species'].str.contains('True').astype(str).astype(bool)
     
#draft_year_list={2007, 2009, 2013}
#draft_month_list={6, 7, 8, 9}
draft_year_list={2007, 2009, 2013}
draft_month_list={6, 7, 8, 9}
frames=[]
for draft_year in draft_year_list:
    data_year=train_data[train_data['year']==draft_year].copy()
    #print(draft_year)
    for draft_month in draft_month_list:
        #print(draft_month)
        train_data_month=data_year[(data_year['month']==draft_month)]
        for dt in iter_days(draft_year, draft_month):
            train_data_day=concat_train_data(train_data_month[(train_data_month['day']==dt.day)])
            if train_data_day.shape[0]>0:
                train_data_day.set_index('Date', inplace=False)
                frames.append(train_data_day)

result = pd.concat(frames, axis=0)
result.reset_index()
result.info()

In [ ]:
#test_truncated_data
def concat_truncated_data(train_data):
    Trap=group_data(train_data, 'Trap', 'Latitude', 'min')
    Longitude=group_data(train_data, 'Trap', 'Longitude', 'min')
    #NumMosquitos=group_data(train_data, 'Trap', 'NumMosquitos', 'sum')
    #WnvPresent=group_data(train_data, 'Trap', 'WnvPresent', 'sum')
    AddressAccuracy=group_data(train_data, 'Trap', 'AddressAccuracy', 'min')
    RESTUANS=group_data(train_data, 'Trap', 'RESTUANS', 'sum')
    PIPIENS=group_data(train_data, 'Trap', 'PIPIENS', 'sum')
    Date_Min=group_data(train_data, 'Trap', 'Date', 'min')
    #Date_Max=group_data(train_data, 'Trap', 'Date', 'max')
    
    frames = [Trap, Longitude, Date_Min, AddressAccuracy, RESTUANS, PIPIENS]
    result = pd.concat(frames, axis=1)
    result.reset_index(inplace=True)
    return result.copy()

test_truncated_data=add_datetime(test_truncated_data)
test_truncated_data['RESTUANS']=test_truncated_data['Species'].str.contains('True').astype(str).astype(bool)
test_truncated_data['PIPIENS']=test_truncated_data['Species'].str.contains('True').astype(str).astype(bool)
test_truncated_data['Trap']=np.power(test_truncated_data['Latitude'], 2)+np.power(test_truncated_data['Longitude'], 2)
#draft_year_list={2007, 2009, 2013}
#draft_month_list={6, 7, 8, 9}

draft_year_list={2011}
draft_month_list={6, 7, 8, 9}
frames=[]
for draft_year in draft_year_list:
    data_year=test_truncated_data[test_truncated_data['year']==draft_year].copy()
    #print(draft_year)
    for draft_month in draft_month_list:
        #print(draft_month)
        train_data_month=data_year[(data_year['month']==draft_month)]
        for dt in iter_days(draft_year, draft_month):
            test_truncated_data_day=concat_truncated_data(train_data_month[(train_data_month['day']==dt.day)])
            if test_truncated_data_day.shape[0]>0:
                test_truncated_data_day.set_index('Date', inplace=False)
                frames.append(test_truncated_data_day)

test_truncated_data_result = pd.concat(frames, axis=0)
test_truncated_data_result.reset_index()
test_truncated_data_result['Date']=np.datetime_as_string(test_truncated_data_result['Date'], unit='D')
#.astype(object)
test_truncated_data_result.info()

In [ ]:
#Merge weather_data and test_data

weather_1=weather_data[weather_data['Station']==1].copy()
weather_1.set_index('Date', inplace=True)
weather_2=weather_data[weather_data['Station']==2].copy()
weather_2.set_index('Date', inplace=True)

weather=weather_1.merge(weather_2,
    #weather_1, left_on='Date', right_on='Date')
    how='inner',
    #on=None, 
    left_on='Date', right_on='Date', 
    #left_index=True, right_index=True,
    #, sort=False,
    suffixes=('_1', '_2'), copy=True
    #, indicator=False, validate=None
)
test_data=result.merge(weather,
    #weather_1, left_on='Date', right_on='Date')
    how='inner',
    #on=None, 
    left_on='Date', right_on='Date', 
    #left_index=True, right_index=True,
    #, sort=False,
    #suffixes=('_x', '_y'), copy=True
    #, indicator=False, validate=None
)

test_truncated_data=test_truncated_data_result.merge(weather,
    #weather_1, left_on='Date', right_on='Date')
    how='inner',
    #on=None, 
    left_on='Date', right_on='Date', 
    #left_index=True, right_index=True,
    #, sort=False,
    #suffixes=('_x', '_y'), copy=True
    #, indicator=False, validate=None
)

#test_truncated_data

In [ ]:
#test_data clear
def clear_dtype(weather_data, drop_dtype_list):
    for col in weather_data:
        if str(weather_data[col].dtypes) in drop_dtype_list:
            try:
                weather_data.drop(
                    weather_data[col].name,  # список колонок або рядків, які потрібно видалити
                    axis=1,  # видалення рядків відбувається аналогічно, щоб видалити саме колонки, вибираємо відповідну вісь
                    inplace=True  # видалення "на місці", без надання нової змінної
                )
                #X_Data=clear_data(X_Data, X_Data[col].name, revers=False)
                print(f'{col}_{weather_data[col].name}_{weather_data[col].dtypes}')
                return weather_data.copy()
            except Exception as _ex:
                print(f"Error", _ex)
                
                
drop_list=['year_2', 'month_2', 'weekday_2', 'day_2']
test_data=clear_data(test_data, drop_list, revers=False)
test_truncated_data=clear_data(test_truncated_data, drop_list, revers=False)
#drop_dtype_list=['bool', 'category', 'int64' ,'int32', 'float64', ]
drop_dtype_list=['category']                
clear_dtype(test_data, drop_dtype_list)
clear_dtype(test_truncated_data, drop_dtype_list)
test_truncated_data.info()

In [ ]:
#X_drop_list=['NumMosquitos', 'WnvPresent', 'RESTUANS', 'PIPIENS', 'Trap', 'Date', 'Latitude', 'Longitude']

X_drop_list=['NumMosquitos', 'WnvPresent', 'Trap', 'Date']
X_Data=clear_data(test_data.copy(), X_drop_list, revers=False)

Y_Data=clear_data(test_data.copy(), X_drop_list, revers=True)
Y_Data=clear_data(Y_Data, ['Trap', 'Date'], revers=False)

X_test_truncated_data=clear_data(test_truncated_data.copy(), X_drop_list, revers=False)


X_drop_list=['Latitude', 'Longitude', 'month_1', 'day_1']
#X_Data=clear_data(test_data.copy(), X_drop_list, revers=True)
#X_test_truncated_data=clear_data(test_data.copy(), X_drop_list, revers=True)

In [ ]:
X_Data.info()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(
    X_Data,  # перший масив, який потрібно розділити
    Y_Data,  # другий масив, який потрібно роздолити
    test_size=0.2,  # розмір меншої групи (20% від загальної кількості даних)
    random_state=42  # фіксація випадкового стану: тепер при перезапуску групи будуть однаковими
)

    # формальність для того, щоб не бачити попередження про зміни на зрізі з таблиці
X_train, X_test = X_train.copy(), X_test.copy()
    # побудуємо модель
mdl = LinearRegression().fit(X_train, y_train)

    # оцінимо точність моделі
train_predictions = mdl.predict(X_train)
test_predictions = mdl.predict(X_test)
truncated_predictions = mdl.predict(X_test_truncated_data)


print('Помилка на навчальному наборі: ', mean_squared_error(y_train, train_predictions))
print('Помилка на тестовому наборі: ', mean_squared_error(y_test, test_predictions))
score = mdl.score(X_test, y_test)
print(score)
#print(mdl.score(X_train, y_test))

In [1]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

# фіксація випадкового стану
np.random.seed(1)
tf.random.set_seed(1)

model = Sequential()  # ініціалізація порожньої моделі

# додамо один перцептрон у модель
model.add(Dense(
    units=1,  # кількість перцептронів
    activation='sigmoid'  # функція активації: перетворення на виході моделі
))

from keras.optimizers import SGD

model.compile(
    optimizer=SGD(learning_rate=0.01),  # алгоритм оптимізації та швидкість навчання
    loss='binary_crossentropy',  # функція втрат (та сама, що в логістичної регресії)
    metrics=['accuracy']  # додаткова метрика якості моделі
)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
continuous_features = set(df.columns) - set(categorical_features)
scaler = MinMaxScaler()
df_norm = df.copy()
df_norm[list(continuous_features)] = scaler.fit_transform(df[list(continuous_features)])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier

# будуємо модель у вигляді дерева
mdl = DecisionTreeClassifier(max_depth=3).fit(X_train, y_train)
# оцінимо якість моделі
predictions = mdl.predict_proba(X_test)
#auc = roc_auc_score(y_test, predictions, average='macro', multi_class='ovr')
print(auc)

In [ ]:
for var_name, coef_value in zip(X_train.columns, mdl.coef_):
    print(var_name)
    
print(f'constant term: {var_name}', mdl.intercept_)

In [ ]:
def centr_map():
    map_ = folium.Map(location=[train_data['Latitude'].mean(),  train_data['Longitude'].mean()],
               tiles="Stamen Terrain", zoom_start=10, control_scale=False)

    Station=folium.FeatureGroup(name='Station')
    Station.add_to(map_)
    Station.add_child(
        folium.Marker([41.995,  -87.933],
                      popup="CHICAGO O'HARE INTERNATIONAL AIRPORT").add_to(map_))
    Station.add_child(
        folium.Marker(
            [41.786,  -88.089163], popup="CHICAGO MIDWAY INTL ARPT").add_to(map_))
    return map_ 

map_=centr_map()



df=spray_data
time_index = list(spray_data['Date'].sort_values().astype('str').unique())


df['Date'] = df['Date'].sort_values(ascending=True)
data = []

for _, d in df.groupby('Date'):
    #print(f'{row}')
    data.append([[row['Latitude'], row['Longitude'], 1] for _, row in d.iterrows()])

hm = plugins.HeatMapWithTime(
    data,
    name="Spray",
    index=time_index,
    auto_play=True,
    min_opacity=0,
    max_opacity=0.7,
    #gradient={0.1:'blue', 0.25:'green', 0.5:'yelow', 0.75:'orange', 1:'red'} 
)
hm.add_to(map_)

folium.LayerControl().add_to(map_)
map_.save(f"sklearn_linear_model.html")


In [ ]:
map_=centr_map()



df=test_data.copy()
time_index = list(test_data['Date'].sort_values().astype('str').unique())


df['Date'] = df['Date'].sort_values(ascending=True)
data = []

for _, d in df.groupby('Date'):
    #print(f'{row}')
    data.append([[row['Latitude'], row['Longitude'], row['NumMosquitos']] for _, row in d.iterrows()])

test_data_hm = plugins.HeatMapWithTime(
    data,
    name="test_data",
    index=time_index,
    auto_play=True,
    min_opacity=0,
    max_opacity=0.7,
    #gradient={0.1:'blue', 0.25:'green', 0.5:'yelow', 0.75:'orange', 1:'red'} 
)
test_data_hm.add_to(map_)





folium.LayerControl().add_to(map_)
map_.save(f"test_data.html")

In [ ]:
test_data

In [ ]:
def plot_markers(): 
    # train_data_point_map
    draft_year_list={2007, 2009, 2013}
    draft_month_list={6, 7, 8, 9}
    geometry = geopandas.points_from_xy(train_data.Longitude, train_data.Latitude)
    geo_df = geopandas.GeoDataFrame(
        train_data, geometry=geometry
    )
    geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry]

    wnvpresentsLayer=folium.FeatureGroup(name='wnvpresents')
    wnvpresentsLayer.add_to(map_)

    # Iterate through list and add a marker for each volcano, color-coded by its type.
    i = 0
    for coordinates in geo_df_list:
        if (geo_df.month[i] == draft_month) and  (geo_df.year[i] == draft_year):
            wnvpresents=int(train_data[(geo_df['Trap'] == geo_df.Trap[i]) & (geo_df['month']==draft_month) 
                       & (geo_df['year']==draft_year)]['WnvPresent'].sum())
            if wnvpresents>0:
                if geo_df.PIPIENS[i] == True:
                    type_color = "green"
                if (geo_df.RESTUANS[i] == True):
                    type_color = "red"
                    # Place the markers with the popup labels and data
                wnvpresentsLayer.add_child(
                    folium.Marker(
                        location=coordinates,
                        popup=geo_df.Trap[i]+ "<br>"
                        +'wnvpresents '+ str(wnvpresents)+ "<br>"+'numMosquitos '+ str(numMosquitos(draft_year, draft_month)),
                        icon=folium.Icon(color="%s" % type_color)
                        )
                )
        i = i + 1

In [ ]:
def plot_datasetes():
    fig = plt.figure()

    plot_mas = test_truncated_data.groupby(['year'], group_keys=True,
                                           dropna=True).agg({'year': ['count']}).to_dict()[('year', 'count')] 
    plt.bar(plot_mas.keys(), plot_mas.values(), color='blue', label='test_truncated_data counts  in year')
    plt.legend()
    plt.grid(True) 



    fig = plt.figure()
    plot_mas = weather_data.groupby(['year'], group_keys=True,
                                           dropna=True).agg({'year': ['count']}).to_dict()[('year', 'count')] 
    plt.bar(plot_mas.keys(), plot_mas.values(), color='pink', label='weather_data counts  in year')
    plt.legend()
    plt.grid(True)   # лінії допоміжної сітки


    fig = plt.figure()
    #fig2 = plt.figure()

    plot_mas = train_data.groupby(['year'], group_keys=True,
                                           dropna=True).agg({'year': ['count']}).to_dict()[('year', 'count')] 
    plt.bar(plot_mas.keys(), plot_mas.values(), color='green', label='train_data counts  in year')
    plt.legend()
    plt.grid(True)   # лінії допоміжної сітки


    fig = plt.figure()
    plot_mas = spray_data.groupby(['year'], group_keys=True,
                                           dropna=True).agg({'year': ['count']}).to_dict()[('year', 'count')] 
    plt.bar(plot_mas.keys(), plot_mas.values(), color='red', label='spray_data counts  in year')
    plt.legend()
    plt.grid(True)   # лінії допоміжної сітки
    
    fig = plt.figure()
    plot_mas = test_truncated_data.groupby(['year'], group_keys=True,
                                           dropna=True).agg({'year': ['count']}).to_dict()[('year', 'count')] 
    plt.bar(plot_mas.keys(), plot_mas.values(), color='red', label='test_truncated_data counts  in year')
    plt.legend()
    plt.grid(True)   # лінії допоміжної сітки

plot_datasetes()

In [ ]:
def bar_chat(_data, grop_list_, agr_colum, agr_func):
    grop_list=[grop_list_]
    agg_func_math = {
        agr_colum: [agr_func]
    }

    plot_mas = _data.groupby(grop_list, group_keys=True, dropna=True).agg(agg_func_math).to_dict()[(agr_colum, agr_func)] 

    fig = plt.figure()
    plt.bar(plot_mas.keys(), plot_mas.values())
    plt.title(f'{grop_list_} {agr_colum} {agr_func} chart')
    
    plt.grid(True)   # лінії допоміжної сітки

In [ ]:
fig = plt.figure()
plot_mas = train_data.groupby(['month'], group_keys=True,
                              dropna=True).agg({'WnvPresent': ['sum']}).to_dict()[('WnvPresent', 'sum')] 
plt.bar(plot_mas.keys(), plot_mas.values(), color='red', label='WnvPresent sum month')
plt.legend()

plt.grid(False)   # лінії допоміжної сітки
#Date
fig = plt.figure()
plot_mas = train_data.groupby(['Date'], group_keys=True,
                              dropna=True).agg({'WnvPresent': ['sum']}).to_dict()[('WnvPresent', 'sum')] 
plt.bar(plot_mas.keys(), plot_mas.values(), color='red', label='WnvPresent sum Date')
plt.legend()

plt.grid(False)   # лінії допоміжної сітки

In [ ]:
train_data_adres.head()

In [ ]:
# Define our key point when map open it
map_ = folium.Map(location=[train_data_adres['Latitude'].mean(),  train_data_adres['Longitude'].mean()],
               tiles="Stamen Terrain", zoom_start=10, control_scale=False)

for la,lo in train_data_adres:
    folium.Marker(
        location=[la,lo],
        icon=folium.Icon(icon_color='white')
    ).add_to(map2)
# Plotting 
map2

In [ ]:
map_ = folium.Map(location=[train_data_adres['Latitude'].mean(),  train_data_adres['Longitude'].mean()],
               tiles="Stamen Terrain", zoom_start=10, control_scale=False)

folium.Marker(train_data_adres['Latitude','Longitude'], popup=train_data_adres['Trap']).add_to(map_)
map_

In [ ]:
#import folium
#Station_1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
#Station_2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level

#m = folium.Map(location=[45.5236, -122.6750])
#m = folium.Map(location=[42.391623, -88.089163], tiles="Stamen Terrain", zoom_start=13)


map_ = folium.Map(location=[spray_data['Latitude'].mean(),  spray_data['Longitude'].mean()],
               tiles="Stamen Terrain", zoom_start=10, control_scale=False)

folium.Marker([41.995,  -87.933], popup="CHICAGO O'HARE INTERNATIONAL AIRPORT").add_to(map_)
folium.Marker([41.786,  -88.089163], popup="CHICAGO MIDWAY INTL ARPT").add_to(map_)

folium.CircleMarker(
    location=[spray_data['Latitude'].mean(),  spray_data['Longitude'].mean()],
    radius=10,
    popup="spray_mean",
    color="#3186cc",
    fill=True,
    fill_color="#3186cc",
).add_to(map_)


#train_data_adres
#for index, location_info in train_data.iterrows():
#    folium.Marker([location_info["Latitude"], location_info["Longitude"]], popup=location_info["NumMosquitos"]).add_to(map_)

map_
